# Large Model Support

Analysing images for classes (image classification) or detecting objects (object detection) is a stable of Deep Learning today.  On your deep learning educational journey, you may have started with simple image classification using low resolution images like the [MNIST](http://yann.lecun.com/exdb/mnist/index.html) numbers dataset below.  These 28 byte square (total of 784 pixels) hand-written numbers are an effective tool to learn about Convolutional Neural Networks (CNN) and how they can be used to classify the images in the dataset into their appropriate numerical correlation.  Images of this size, along with the typical deep learning models, can *very* easily fit inside of GPU memory.

### <center> MNIST Dataset</center>
![MNIST](https://upload.wikimedia.org/wikipedia/commons/2/27/MnistExamples.png)

Today, however, the use of Deep Learning to identify items of greater visual density (i.e. higher resolution) is key.  Pictures from personal devices, security cameras, medical imaging equipment and more produce some very large high resolution images.  These higher resolution images often must be transformed into lower resolutions in order to fit into the CNN models running on GPU memory.  This typically reduces accuracy and/or means data loss.

Imagine if your doctor used your MRI images to discover the existence of an anomaly, but the images were reduced to 100x100 pixel images !!  (some of you may remember seeing pictures on your first flip phone!  Others might want to ask your parents...)  You would *not* be comfortable with that!

An example brain scan analysed by a subject matter expert and a deep learning model:

![Brain scan illustration](https://github.com/smatzek/3DUnetCNN/raw/tflmsv2/doc/tumor_segmentation_illusatration.gif)

 

IBM's [TensorFlow Large Model Support](https://developer.ibm.com/linuxonpower/2018/12/19/powerai-tensorflow-large-model-support-multi-gpu-competitive-comparison/), or TFLMS, enables usage of higher resolution datasets, larger models and/or larger batch sizes by allowing the system memory to be used in conjunction with the GPU memory. TFLMS modifies the TensorFlow graph prior to training by injecting swap nodes that will swap tensors in and out of GPU memory to system memory.

This lab uses TFLMS v2 that comes as part of [Watson Machine Learning Community Edition v1.6.1](https://www.ibm.com/support/knowledgecenter/en/SS5SF7_1.6.1/navigation/welcome.html) 's <code>tensorflow-large-model-support</code> conda package.  Not to be confused with TensorFlow V2.0 (available publicly as of September 30, 2019), TFLMS v2 is based on TensorFlow V1.14 at the time of this lab.

## Lab Environment


In our lab you are using an IBM POWER9-based  [Power System AC922](https://www.ibm.com/us-en/marketplace/power-systems-ac922) server with two or four NVIDIA Volta V100 GPUs attached to the system bus via NVLINK 2.0.
![AC922 with NVLink](https://developer.ibm.com/linuxonpower/wp-content/uploads/sites/105/2018/12/tflms.png)

The NVLINK high-speed bus technology is available on x86 computers with NVIDIA GPUs, but <b>only</b> for GPU to GPU connections.  IBM's POWER9 CPU has NVLINK 2.0 connections directly on the CPU to allow for high-speed bandwidth from the GPU to system memory.

In [ ]:
# View the GPU devices, NVIDIA Driver version, NVIDIA CUDA version and more.  You can see these are 16GB GPUs
!/usr/bin/nvidia-smi

**Where's the other GPUs?!**

Because we are sharing this server among 4 students, each student is running in a Docker container with access to only a single GPU.

| <b>Team | GPU</b> |
| ---- | --- |
| team01, team05, team09, team13, team17 | 0 |
| team02, team06, team10, team14, team18 | 1 |
| team03, team07, team11, team15, team19 | 2 |
| team04, team08, team12, team16, team20 | 3 |

If you were running in bare-metal, you can utilize the <code>CUDA_VISIBLE_DEVICES</code> environment variable to specify the GPU(s) in use.

In [ ]:
# Now to view the system memory (these systems have 512GB of phyxical memory)
!free -h

Hmm, that looks like **more** than 512GB!

Well, thanks to NVLINK 2.0 connection on the system bus and the associated ability for the GPU to access CPU memory in a simple manner, the total GPU memory has been added to the CPU memory with the Linux command run above.

Since the lab environment has, at most, four students per physical server, we are attempting a fair split of standard system memory by setting the following LMS environment variables (if you want, use the **env** command in the terminal window to see for yourself):

<code>TF_CUDA_HOST_MEM_LIMIT_IN_MB=131072
    
TF_GPU_HOST_MEM_LIMIT_IN_MB=131072</code>

During the development of this lab, it was discovered that the default LMS simulator memory ratio of 0.9 was too high and a reduction to 0.8 (80%) was needed to complete auto tuning (or simulation).  Auto tuning excludes a configurable portion of GPU memory during its simulated training to allow for memory overhead related to things like garbage collection, metrics, temporary memory allocations within operations, or cross GPU communication.

<code>TF_LMS_SIMULATOR_MEM_RATIO=0.8</code>

### 3D U-net CNN

[3D U-net CNN](https://github.com/ellisdg/3DUnetCNN) is a three dimensional convolutional neural network designed for medical image segementation.

Originally designed after [this paper](http://lmb.informatik.uni-freiburg.de/Publications/2016/CABR16/cicek16miccai.pdf) on volumetric segmentation with a 3D U-Net. The code was written to be trained using the [BRatS](https://www.med.upenn.edu/sbia/brats2017.html) data set for brain tumors, but it can be easily modified to be used in other 3D applications.

The original source for this code is available in github at:
https://github.com/ellisdg/3DUnetCNN

and has been enabled to utilize IBM's Large Model Support, also available from github at:
https://github.com/smatzek/3DUnetCNN

The BRatS 2017 dataset utilizes multi-institutional pre-operative MRI scans and focuses on the segmentation of intrinsically heterogeneous (in appearance, shape, and histology) brain tumors, namely gliomas.

Anyone wanting a simple example of using TFLMS v2 can utilize a Keras-based ResNet50 model that is included in Watson ML Community Edition and modified to support LMS.  The [ResNet50](https://www.kaggle.com/keras/resnet50) model demonstrates LMS using auto-generated data (i.e. not external datasets like BRatS). It provides a convenient way to test out the basics of LMS.

## Training without LMS
Each image size specified in the training run utilizes a separate data file that is generated automatically (if not specified) from the processed source data files.  This lab will not attempt to get to high levels of accuracy since that requires running through many epochs, taking more time than available.  The lab also utilizes pre-generated data files since data file generate also takes an extended time.

### 3DUnetCNN without LMS, image size of 64
Rather than include the entirety of the source code inside of the notebook, we're just running via the command-line for simplicity.  Initially we will run a training job using with an image size of 64, which represents a 64x64x64 [voxel](https://en.wikipedia.org/wiki/Voxel) cube under the covers.  Also we're initially chosing 10 epochs since this typically finishes in under 2 minutes.

In [ ]:
!python $HOME/brats/train_isensee2017.py --epochs 10 --data_file_path /root/brats_data_64.h5 \
--image_size 64

Increasing the image size adds time to each epoch.  If we jump from 64 to 144 for the image size, but just two epochs we see the time per epoch go up!

### 3DUnetCNN without LMS, image size of 144

In [ ]:
!python $HOME/brats/train_isensee2017.py --epochs 2 --data_file_path /root/brats_data_144.h5 \
--image_size 144

So this completed without problems, but took a longer period of time. We are not analyzing/comparing the actual loss values since this lab uses minimal epochs due to time constraints. Increasing the image size from 144 to 192 should provide some different results...

### 3DUnetCNN without LMS, image size of 192

In [ ]:
!python $HOME/brats/train_isensee2017.py --epochs 2 --data_file_path /root/brats_data_192.h5 \
--image_size 192

During this run, it will likely fail with an error message such as **Resource exhausted: OOM when allocating tensor with shape[1,16,192,192,192]** somewhere in the output (and yes, it should generate a LOT more output).

So between the image size and the size of the CNN we have exhausted the memory of the GPU!

Since the code has already been modified to utilize LMS, let's use the flag to enable it.

### 3DUnetCNN with LMS, image size of 192

This run will take longer than previous runs.  Before the actual training begins, LMS will run through simulated training to determine initial optimal LMS tuning parameters.  This process will take 3-4 minutes in this lab.

In [ ]:
## So lets enable LMS
!python $HOME/brats/train_isensee2017.py --epochs 2 --data_file_path /root/brats_data_192.h5 \
--image_size 192 --lms

If you are familiar, you can use a second terminal session (the terminal from withing Jupyter won't work, you'll have to use the host OS) accessed via SSH and use nmon to monitor the GPU utilization during your training run.  The "a" flag in this compiled version of nmon detects the CUDA libraries and is able to show GPU speed in MHz, processor/memory utilization, temperature and power usage.
><code>
┌nmon─16f──────[H for help]───Hostname=ac922X───────Refresh= 2secs ───22:00.00───┐
│ NVIDIA GPU Accelerator ────────────────────────────────────────────────────────
│ Driver Version:418.39    NVML Version: 10.418.39
│ GPU          GPU-Utilisation  Temperature  Power-Use  Name
│ No. GPU-MHz  Processor-Memory  Centigrade      Watts
│   0     135        0%       0%     36        37.21    Tesla V100-SXM2-16GB
│   1     135        0%       0%     39        38.68    Tesla V100-SXM2-16GB
│ <b>  2    1530      100%      79%     46       223.55    Tesla V100-SXM2-16GB</b>
│   3     135        0%       0%     38        37.21    Tesla V100-SXM2-16GB
│────────────────────────────────────────────────────────────────────────────────

Near the beginning of this output, you'll see a TensorFlow graph analysis similar to the following (edited down for cleaner output):

<code>I1001 00:45:50.865546 lms.py:1198] [LMS][0] Editing model for LMS
I1001 00:45:52.052714 lms.py:1198] [LMS][0] The graph has 14154 vertices and 18582 edges.
I1001 00:45:52.052905 lms.py:1198] [LMS][0] The graph has 94.64 MiB of learning parameters.
I1001 00:45:52.052978 lms.py:1198] [LMS][0] The largest GPU operation is training/gradients/AddN_435 consuming 2.11 GiB
I1001 00:45:55.618135 lms.py:1198] [LMS][0] Original categorized topological sort has 1300 levels.
I1001 00:45:56.432480 lms.py:1198] [LMS][0] Searching values for parameters: swapout_threshold, swapin_ahead, swapin_groupby and sync_mode. </code>

At this stage, the LMS simulator will enter auto tune mode to determine good parameters for the swapping of tensors.

<code>I1001 00:45:57 lms.py:1198] [LMS][0] [Simulator] Available memory for simulation: 11.59 GiB (memory ratio: 0.8)
I1001 00:46:01 lms.py:1198] [LMS][0] [Simulator] Found a parameter set: swapout_threshold 1, swapin_ahead 1, swapin_groupby 0, sync_mode 0
I1001 00:46:04 lms.py:1198] [LMS][0] [Simulator] Found a parameter set: swapout_threshold 325, swapin_ahead 1, swapin_groupby 0, sync_mode 0
I1001 00:46:10 lms.py:1198] [LMS][0] [Simulator] Found a parameter set: swapout_threshold 345, swapin_ahead 1, swapin_groupby 0, sync_mode 0
I1001 00:46:12 lms.py:1198] [LMS][0] [Simulator] Found a parameter set: swapout_threshold 355, swapin_ahead 1, swapin_groupby 0, sync_mode 0
I1001 00:47:25 lms.py:1198] [LMS][0] [Simulator] Found a parameter set: swapout_threshold 355, swapin_ahead 162, swapin_groupby 0, sync_mode 0
I1001 00:47:50 lms.py:1198] [LMS][0] [Simulator] Found a parameter set: swapout_threshold 355, swapin_ahead 202, swapin_groupby 0, sync_mode 0
I1001 00:48:02 lms.py:1198] [LMS][0] [Simulator] Found a parameter set: swapout_threshold 355, swapin_ahead 222, swapin_groupby 0, sync_mode 0
I1001 00:48:18 lms.py:1198] [LMS][0] [Simulator] Found a parameter set: swapout_threshold 355, swapin_ahead 232, swapin_groupby 0, sync_mode 0
I1001 00:48:46 lms.py:1198] [LMS][0] [Simulator] Found a parameter set: swapout_threshold 355, swapin_ahead 234, swapin_groupby 0, sync_mode 0
I1001 00:49:02 lms.py:1198] [LMS][0] [Simulator] Found a parameter set: swapout_threshold 355, swapin_ahead 235, swapin_groupby 0, sync_mode 0
. . .
I1001 00:49:34 lms.py:1198] [LMS][0] LMS will use the latest parameter set found by Simulator for the best performance. However, if you encounter an out-of-memory error, please manually use the previous parameter set found by Simulator.
I1001 00:49:34 lms.py:1198] [LMS][0]   sync_mode: 0 (Asynchronous memory copy between host and device)
I1001 00:49:34 lms.py:1198] [LMS][0]   swapout_threshold: 355
I1001 00:49:34 lms.py:1198] [LMS][0]   swapin_ahead: 235 
I1001 00:49:34 lms.py:1198] [LMS][0]   swapin_groupby: 1300 
I1001 00:49:49 lms.py:1198] [LMS][0] Added 512 operations to the model (256 swap-out operations (10.62 GiB) and 256 swap-in operations (10.62 GiB))
I1001 00:49:50 lms.py:1198] [LMS][0] Editing model for LMS, took: 238328.46546173096 ms
. . .
Epoch 1/2
37/37 [==============================] - 124s 3s/step - loss: -0.0401 - val_loss: -0.0368
Epoch 2/2
37/37 [==============================] - 78s 2s/step - loss: -0.0437 - val_loss: -0.0360

</code>


Once a deep learning model with selected image size is run through with an epoch or two for validation, you can re-run the same job specifying the LMS parameters and skipping the simulation overhead.

In [ ]:
!python $HOME/brats/train_isensee2017.py --epochs 2 --data_file_path /root/brats_data_192.h5 \
--image_size 192 --lms --sync_mode 0 --swapout_threshold 355 --swapin_ahead 235 --swapin_groupby 1300

You can see that this run takes approximately 4 minutes less (without the simulation steps) and the time per epoch should be similar to the previous training run.

If LMS is required for inferencing, prediction, and evaluation, then manual tuning techniques should be used.  The [Knowledge Center](https://www.ibm.com/support/knowledgecenter/en/SS5SF7_1.6.1/navigation/wmlce_getstarted_tflmsv2.html) has detailed information on parameters and usage.

### 3DUnetCNN with LMS, image size of 256

As a last example, we can run with an image size of 256.  This is at least **64 times** the amount of data per image compared to the first run with image size of 64.

This last run is optional.

In [ ]:
!python $HOME/brats/train_isensee2017.py --epochs 3 --data_file_path /root/brats_data_256.h5 \
--image_size 256 --lms --sync_mode 0 --swapout_threshold 106 --swapin_ahead 5 --swapin_groupby 650

### Additional Resources

In addition to the hyperlinks in the lab, there are various blogs and articles about LMS including a [landing page for all things LMS](https://developer.ibm.com/linuxonpower/2019/06/11/tensorflow-large-model-support-resources/).


* [Performance of 3DUnet Multi GPU Model for Medical Image Segmentation using TensorFlow Large Model Support](https://developer.ibm.com/linuxonpower/2018/12/19/performance-of-3dunet-multi-gpu-model-for-medical-image-segmentation-using-tensorflow-large-model-support/)

* [TensorFlow Large Model Support Case Study with 3D Image Segmentation](https://developer.ibm.com/linuxonpower/2018/07/27/tensorflow-large-model-support-case-study-3d-image-segmentation/)